### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**DOWNLOADING LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [5]:
#@ DOWNLOADING THE LIBRARIES AND DEPENDENCIES:
# !pip install -U d2l
from d2l import torch as d2l

import os
import torch     
from torch import nn                                
from torch.nn import functional as F

**GETTING THE DATASET:**
- I have used google colab for this notebook so the process of downloading and reading the data might be different in other platforms. I will use Stanford Natural Language Inference Corpus for this notebook. The SNLI Corpus is a collection of over 500000 labeled english pairs.

In [7]:
#@ GETTING THE DATASET: 
batch_size, num_steps = 256, 50                                          # Initializing Parameters. 
train_iter, test_iter, vocab = d2l.load_data_snli(batch_size, num_steps) # Initializing Data Iterations. 

read 549367 examples
read 9824 examples


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### **ATTENDING CLASS:**
- I will align words in one text sequnce to each word in other sequence. I will implement soft alignment using attention mechanism. 

In [8]:
#@ IMPLEMENTING MULTILAYER PERCEPTRON: 
def mlp(num_inputs, num_hiddens, flatten):                 # Function for MLP.
  net = []                                                 # Initializing Lists.
  net.append(nn.Dropout(0.2))                              # Initializing Dropout Layer. 
  net.append(nn.Linear(num_inputs, num_hiddens))           # Initializing Linear Layer. 
  net.append(nn.ReLU())                                    # Initializing RELU Activation. 
  if flatten:
    net.append(nn.Flatten(start_dim=1))                    # Initializing Flatten Layer. 
  net.append(nn.Dropout(0.2))                              # Initializing Dropout Layer. 
  net.append(nn.Linear(num_hiddens, num_hiddens))          # Initializing Linear Layer. 
  net.append(nn.ReLU())                                    # Initializing RELU Activation. 
  if flatten:
    net.append(nn.Flatten(start_dim=1))                    # Initializing Flatten Layer. 
  return nn.Sequential(*net)                               # Initializing Sequential API. 

- I will define the `Attend Class` to compute the soft alignment of the hypotheses `beta` with input premises and soft alignment of premises `alpha` with input hypotheses. 

In [9]:
#@ IMPLEMENTATION OF ATTEND CLASS: 
class Attend(nn.Module):                                            # Initializing Attend Class. 
  def __init__(self, num_inputs, num_hiddens, **kwargs):            # Initializing Constructor Function. 
    super(Attend, self).__init__(**kwargs)
    self.f = mlp(num_inputs, num_hiddens, flatten=False)            # Initialization of MLP. 
  
  def forward(self, A, B):                                          # Forward Propagation Function. 
    f_A = self.f(A)                                                 # Implementation of MLP. 
    f_B = self.f(B)                                                 # Implementation of MLP. 
    e = torch.bmm(f_A, f_B.permute(0, 2, 1))                        # Implementation of Matrix Multiplication. 
    beta = torch.bmm(F.softmax(e, dim=-1), B)                       # Implementation of Softmax. 
    alpha = torch.bmm(F.softmax(e.permute(0, 2, 1), dim=-1), A)     # Implementation of Softmax. 
    return beta, alpha